In [5]:
import os
import gzip
import shutil

In [8]:
def fixBadZipfile(zipFile):
 f = open(zipFile, 'r+b')
 data = f.read()
 pos = data.find(b'\x50\x4b\x05\x06') # End of central directory signature
 if (pos > 0):
     print(f"Truncating file at location {pos + 22}.")
     f.seek(pos + 22)   # size of 'ZIP end of central directory record'
     f.truncate()
     f.close()
 else:
     print("raise error, file is truncated")



Truncating file at location 2452590457.


In [3]:

def copy_and_unzip(from_path, to_path):

    if "training" in from_path:
        train_or_Test = "train"
    else:
        train_or_Test = "test"

    for patient in os.listdir(from_path):
        patient_path = os.path.join(from_path, patient)

        if not os.path.isdir(patient_path):
            continue

        where_to = os.path.join(to_path, train_or_Test, patient)
        os.makedirs(where_to, exist_ok=True)  # Ensure output directory exists

        for f in os.listdir(patient_path):
            input_file = os.path.join(patient_path, f)

            #Skip directories (prevents IsADirectoryError)
            if os.path.isdir(input_file):
                continue

            #Skip specific files
            if f.endswith('md'):
                continue  # Skip mandatory_citation.md

            #Process .nii.gz files
            if f.endswith('.nii.gz'):
                output_file = os.path.join(where_to, f[:-3])  # Remove .gz
                with gzip.open(input_file, 'rb') as f_in:
                    with open(output_file, 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)

            #Copy other files normally
            else:
                output_file = os.path.join(where_to, f)
                with open(input_file, 'rb') as f_in:
                    with open(output_file, 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)

    print("Extraction completed successfully!")


### Place the ACDC.zip file somewhere in your directory and use that path for the following cell:

In [9]:
import zipfile as zf

zip_path="../datasets/ACDC.zip"


fixBadZipfile(zip_path)
filess = zf.ZipFile(zip_path, 'r')
filess.extractall('raw_dataset')
filess.close()

### Use your own data paths to the testing or training folder of ACDC for from_path and a desired location as to_path
### the desired location directory has to be created prior by you and include two subdirectories: train and test

In [7]:
from_path = "/home/jovyan/DL Medical Image/dataset/ACDC/database/testing"
to_path = "/home/jovyan/DL Medical Image/dataset/dataset_extracted"

copy_and_unzip(from_path, to_path)

Extraction completed successfully!
